# Generative AI Use Case: Summarize Dialogue

Welcome to the practical side of this course. In this lab you will do the dialogue summarization task using generative AI. You will explore how the input text affects the output of the model, and perform prompt engineering to direct it towards the task you need. By comparing zero shot, one shot, and few shot inferences, you will take the first step towards prompt engineering and see how it can enhance the generative output of Large Language Models.

# Table of Contents

- [ 1 - Set up Kernel and Required Dependencies](#1)
- [ 2 - Summarize Dialogue without Prompt Engineering](#2)
- [ 3 - Summarize Dialogue with an Instruction Prompt](#3)
  - [ 3.1 - Zero Shot Inference with an Instruction Prompt](#3.1)
  - [ 3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5](#3.2)
- [ 4 - Summarize Dialogue with One Shot and Few Shot Inference](#4)
  - [ 4.1 - One Shot Inference](#4.1)
  - [ 4.2 - Few Shot Inference](#4.2)
- [ 5 - Generative Configuration Parameters for Inference](#5)


<a name='1'></a>
## 1 - Set up Kernel and Required Dependencies

First, check that the correct kernel is chosen.

<img src="images/kernel_set_up.png" width="300"/>

You can click on that (top right of the screen) to see and check the details of the image, kernel, and instance type.

<img src="images/w1_kernel_and_instance_type.png" width="600"/>




Now install the required packages to use PyTorch and Hugging Face transformers and datasets.



In [6]:
import pandas as pd 
# import nltk
import sqlite3
# from nltk.translate.bleu_score import sentence_bleu

# programming languages that CodeT5 supports
languages = ['Python', 'Java', 'JavaScript', 'PHP', 'Ruby', 'Go', 'C', 'C#']
# languages = ['Java']

conn = sqlite3.connect('/Users/guru/research/FixMe/data/FixMe-v2.db')

df_cve = pd.read_sql_query("SELECT * FROM cve;", conn)
df_repository = pd.read_sql_query("SELECT * FROM repository;", conn)
df_hunk = pd.read_sql_query("SELECT * FROM hunk_collection;", conn)
df_patch = pd.read_sql_query("SELECT * FROM patch_collection;", conn)
print(f'Original hunks shape: {df_hunk.shape}')

df = df_hunk[df_hunk.programming_language.isin(languages)].reset_index(drop=True)
df = df[['code_before', 'code_after', 'programming_language']]
print(f'Shape of {languages} hunks: {df.shape}')
df.head()

Original hunks shape: (44059, 19)
Shape of ['Python', 'Java', 'JavaScript', 'PHP', 'Ruby', 'Go', 'C', 'C#'] hunks: (27461, 3)


,code_before,code_after,programming_language
0,*********************************************...,*********************************************...,Java
1,private boolean exractAndLoad(ArrayList<St...,private boolean exractAndLoad(ArrayList<St...,Java
2,\n // Fall back to extr...,\n // Fall back to extr...,Java
3,return libName;\n }\n\n private ...,return libName;\n }\n\n private ...,Java
4,* - http://www.gnu.org/copyleft/gpl.html\n *...,* - http://www.gnu.org/copyleft/gpl.html\n *...,JavaScript


In [ ]:
def filter_patches(df_patch, max_hunks_per_file=2):
    """Filter URLs with counts less than max_hunks_per_file"""
    # Calculate value counts of 'url' column
    url_counts = df_patch['url'].value_counts()
    urls_less_than_two = url_counts[url_counts <= max_hunks_per_file].index.tolist()
    df_filter = df_patch[df_patch.url.isin(urls_less_than_two)]

    print(f'Shape of filtered patch data: {df_filter.shape}')
    return df_filter

def filter_hunks(df_hunk, df_patch):
    """Filter hunks that are not in the filtered patches"""
    df_hunk = df_hunk[df_hunk.file.isin(df_patch.file)]
    print(f'Shape of filtered hunk data: {df_hunk.shape}')
    return df_hunk

max_hunks_per_file = 1
df_patch_filter = filter_patches(df_patch, max_hunks_per_file)
df_hunk_filter = filter_hunks(df_hunk, df_patch_filter)

df_hunk_filter = df_hunk_filter[df_hunk_filter.programming_language.isin(languages)].reset_index(drop=True)
print(f'Shape of filtered hunks (pl-list): {df_hunk_filter.shape}')

In [ ]:
index = 789
prompt = """
Original code:

""" + df_hunk_filter.code_before[index] + """

Modified code:
    
    """ + df_hunk_filter.code_after[index] + """

    What is the modification related in the original code?
    """

print(prompt)

In [ ]:
import re

def replace_whitespaces_N21(code):
    # Use regular expression to replace multiple whitespaces with a single space
    return re.sub(r'\s+', ' ', code).strip()


# Replace multiple whitespaces with a single space
formatted_original_code = replace_whitespaces_N21(original_code)
formatted_new_code = replace_whitespaces_N21(new_code)

match = formatted_original_code == formatted_new_code
print(f'Match: {match}')

In [ ]:
def show_tables(conn):
    # List all tables in the database
    query = "SELECT name FROM sqlite_master WHERE type='table';"
    tables = pd.read_sql_query(query, conn)
    print("Tables in the database:", tables['name'].tolist())

show_tables(conn)

In [ ]:
mask = df_patch.message.value_counts()
mask = mask[mask <= 1].index
df_patch_1 = df_patch[df_patch.message.isin(mask)].reset_index(drop=True)
print(f'Original patches shape: {df_patch_1.shape}')

In [ ]:
df_repository.cveId

In [ ]:
df_patch_1

Load the datasets, Large Language Model (LLM), tokenizer, and configurator. Do not worry if you do not understand yet all of those components - they will be described and discussed later in the notebook.

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

<a name='2'></a>
## 2 - Summarize Dialogue without Prompt Engineering

In this use case, you will be generating a summary of a dialogue with the pre-trained Large Language Model (LLM) FLAN-T5 from Hugging Face. The list of available models in the Hugging Face `transformers` package can be found [here](https://huggingface.co/docs/transformers/index). 

Let's upload some simple dialogues from the [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum) Hugging Face dataset. This dataset contains 10,000+ dialogues with the corresponding manually labeled summaries and topics. 

In [ ]:
huggingface_dataset_name = "knkarthick/dialogsum"

dataset = load_dataset(huggingface_dataset_name)

In [ ]:
from datasets import Dataset, DatasetDict

# Split the DataFrame into train, validation, and test sets
train_df = df.iloc[:800]
validation_df = df.iloc[800:900]
test_df = df.iloc[900:]

# Convert DataFrame to list of dictionaries
def df_to_dicts(df):
    return [
        {
            'id': i,
            'dialogue': row['code_before'],
            'summary': row['code_after'],
            'topic': '', # Optional field
        }
        for i, row in df.iterrows()
    ]

# Create Dataset objects
train_dataset = Dataset.from_dict({'id': list(train_df.index), 'dialogue': train_df['code_before'], 'summary': train_df['code_after'], 'topic': [''] * len(train_df)})
validation_dataset = Dataset.from_dict({'id': list(validation_df.index), 'dialogue': validation_df['code_before'], 'summary': validation_df['code_after'], 'topic': [''] * len(validation_df)})
test_dataset = Dataset.from_dict({'id': list(test_df.index), 'dialogue': test_df['code_before'], 'summary': test_df['code_after'], 'topic': [''] * len(test_df)})

# Create DatasetDict with the desired format
mydataset = DatasetDict({
    'train': train_dataset,
    'validation': validation_dataset,
    'test': test_dataset
})
mydataset

In [ ]:
example_indices = [40, 200]

dash_line = '-'.join('' for x in range(50))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print('INPUT DIALOGUE:')
    print(mydataset['test'][index]['dialogue'])
    print(dash_line)
    print('BASELINE HUMAN SUMMARY:')
    print(mydataset['test'][index]['summary'])
    print(dash_line)
    print()

Print a couple of dialogues with their baseline summaries.

Load the [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5), creating an instance of the `AutoModelForSeq2SeqLM` class with the `.from_pretrained()` method. 

In [ ]:
# model_name='google/flan-t5-base'
model_name='Salesforce/codet5-base'

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

To perform encoding and decoding, you need to work with text in a tokenized form. **Tokenization** is the process of splitting texts into smaller units that can be processed by the LLM models. 

Download the tokenizer for the FLAN-T5 model using `AutoTokenizer.from_pretrained()` method. Parameter `use_fast` switches on fast tokenizer. At this stage, there is no need to go into the details of that, but you can find the tokenizer parameters in the [documentation](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoTokenizer).

Test the tokenizer encoding and decoding a simple sentence:

In [ ]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration

tokenizer = RobertaTokenizer.from_pretrained("Salesforce/codet5p-220m")
model = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5p-220m")


In [ ]:
text = "def greet(user): print(f'hello <extra_id_0>!')"
# text = "def add(a, b): \n int sum= a + b \n return sum"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=8)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
# this prints "{user.username}"

Now it's time to explore how well the base LLM summarizes a dialogue without any prompt engineering. **Prompt engineering** is an act of a human changing the **prompt** (input) to improve the response for a given task.

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = mydataset['test'][index]['dialogue']
    summary = mydataset['test'][index]['summary']
    
    inputs = tokenizer(dialogue, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

You can see that the guesses of the model make some sense, but it doesn't seem to be sure what task it is supposed to accomplish. Seems it just makes up the next sentence in the dialogue. Prompt engineering can help here.

<a name='3'></a>
## 3 - Summarize Dialogue with an Instruction Prompt

Prompt engineering is an important concept in using foundation models for text generation. You can check out [this blog](https://www.amazon.science/blog/emnlp-prompt-engineering-is-the-new-feature-engineering) from Amazon Science for a quick introduction to prompt engineering.

<a name='3.1'></a>
### 3.1 - Zero Shot Inference with an Instruction Prompt

In order to instruct the model to perform a task - summarize a dialogue - you can take the dialogue and convert it into an instruction prompt. This is often called **zero shot inference**.  You can check out [this blog from AWS](https://aws.amazon.com/blogs/machine-learning/zero-shot-prompting-for-the-flan-t5-foundation-model-in-amazon-sagemaker-jumpstart/) for a quick description of what zero shot learning is and why it is an important concept to the LLM model.

Wrap the dialogue in a descriptive instruction and see how the generated text will change:

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = mydataset['test'][index]['dialogue']
    summary = mydataset['test'][index]['summary']

    prompt = f"""
Vulnerable C program:

{dialogue}

Patch of the program:
    """

    # Input constructed prompt instead of the dialogue.
    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=100,
        )[0], 
        skip_special_tokens=True
    )
    
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)    
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

In [ ]:
import evaluate
# summary
# output
bleu = evaluate.load('bleu')
bleu.compute(predictions=[output], references=[summary])

In [ ]:
from evaluate import load
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

code_eval = load("code_eval")
test_cases = [summary]
candidates = [[output]]
pass_at_k, results = code_eval.compute(references=test_cases, predictions=candidates, k=[1 ,5, 10])
print(pass_at_k)

In [ ]:
from evaluate import load
super_glue_metric = load('super_glue', 'copa') 
predictions = [0, 1]
references = [0, 0]
results = super_glue_metric.compute(predictions=predictions, references=references)
print(results)

This is much better! But the model still does not pick up on the nuance of the conversations though.

**Exercise:**

- Experiment with the `prompt` text and see how the inferences will be changed. Will the inferences change if you end the prompt with just empty string vs. `Summary: `?
- Try to rephrase the beginning of the `prompt` text from `Summarize the following conversation.` to something different - and see how it will influence the generated output.

<a name='3.2'></a>
### 3.2 - Zero Shot Inference with the Prompt Template from FLAN-T5

Let's use a slightly different prompt. FLAN-T5 has many prompt templates that are published for certain tasks [here](https://github.com/google-research/FLAN/tree/main/flan/v2). In the following code, you will use one of the [pre-built FLAN-T5 prompts](https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py):

In [ ]:
for i, index in enumerate(example_indices):
    dialogue = mydataset['test'][index]['dialogue']
    summary = mydataset['test'][index]['summary']
        
    prompt = f"""
Vulnerable C program:

{dialogue}

What is the patch of the program?
"""

    inputs = tokenizer(prompt, return_tensors='pt')
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"], 
            max_new_tokens=50,
        )[0], 
        skip_special_tokens=True
    )

    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')

Notice that this prompt from FLAN-T5 did help a bit, but still struggles to pick up on the nuance of the conversation. This is what you will try to solve with the few shot inferencing.

<a name='4'></a>
## 4 - Summarize Dialogue with One Shot and Few Shot Inference

**One shot and few shot inference** are the practices of providing an LLM with either one or more full examples of prompt-response pairs that match your task - before your actual prompt that you want completed. This is called "in-context learning" and puts your model into a state that understands your specific task.  You can read more about it in [this blog from HuggingFace](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

<a name='4.1'></a>
### 4.1 - One Shot Inference

Let's build a function that takes a list of `example_indices_full`, generates a prompt with full examples, then at the end appends the prompt which you want the model to complete (`example_index_to_summarize`).  You will use the same FLAN-T5 prompt template from section [3.2](#3.2). 

In [ ]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = mydataset['test'][index]['dialogue']
        summary = mydataset['test'][index]['summary']
        
        # The stop sequence '{summary}\n\n\n' is important for FLAN-T5. Other models may have their own preferred stop sequence.
        prompt += f"""
Vulerable C program:

{dialogue}

What is the patch of the program?

{summary}

"""
    
    dialogue = mydataset['test'][example_index_to_summarize]['dialogue']
    
    prompt += f"""
Vulerable C program:

{dialogue}

What is the patch of the program?
"""
        
    return prompt

Construct the prompt to perform one shot inference:

In [ ]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

Now pass this prompt to perform the one shot inference:

In [ ]:
summary = mydataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ONE SHOT:\n{output}')

<a name='4.2'></a>
### 4.2 - Few Shot Inference

Let's explore few shot inference by adding two more full dialogue-summary pairs to your prompt.

In [ ]:
example_indices_full = [30, 10, 150]
example_index_to_summarize = 300

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

Now pass this prompt to perform a few shot inference:

In [ ]:

dialogue = mydataset['test']['dialogue'][example_index_to_summarize]
summary = mydataset['test']['summary'][example_index_to_summarize]

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=100,
    )[0], 
    skip_special_tokens=True
)
print(dash_line)
print(f'Prompt:\n{dialogue}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')

In this case, few shot did not provide much of an improvement over one shot inference.  And, anything above 5 or 6 shot will typically not help much, either.  Also, you need to make sure that you do not exceed the model's input-context length which, in our case, if 512 tokens.  Anything above the context length will be ignored.

However, you can see that feeding in at least one full example (one shot) provides the model with more information and qualitatively improves the summary overall.

**Exercise:**

Experiment with the few shot inferencing.
- Choose different dialogues - change the indices in the `example_indices_full` list and `example_index_to_summarize` value.
- Change the number of shots. Be sure to stay within the model's 512 context length, however.

How well does few shot inferencing work with other examples?

<a name='5'></a>
## 5 - Generative Configuration Parameters for Inference

You can change the configuration parameters of the `generate()` method to see a different output from the LLM. So far the only parameter that you have been setting was `max_new_tokens=50`, which defines the maximum number of tokens to generate. A full list of available parameters can be found in the [Hugging Face Generation documentation](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig). 

A convenient way of organizing the configuration parameters is to use `GenerationConfig` class. 

**Exercise:**

Change the configuration parameters to investigate their influence on the output. 

Putting the parameter `do_sample = True`, you activate various decoding strategies which influence the next token from the probability distribution over the entire vocabulary. You can then adjust the outputs changing `temperature` and other parameters (such as `top_k` and `top_p`). 

Uncomment the lines in the cell below and rerun the code. Try to analyze the results. You can read some comments below.

In [ ]:
# generation_config = GenerationConfig(max_new_tokens=50)
# generation_config = GenerationConfig(max_new_tokens=10)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.1)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=2.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0], 
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

Comments related to the choice of the parameters in the code cell above:
- Choosing `max_new_tokens=10` will make the output text too short, so the dialogue summary will be cut.
- Putting `do_sample = True` and changing the temperature value you get more flexibility in the output.

As you can see, prompt engineering can take you a long way for this use case, but there are some limitations. Next, you will start to explore how you can use fine-tuning to help your LLM to understand a particular use case in better depth!

# # Fine-Tune a Generative AI Model for Dialogue Summarization

In [ ]:
%pip install -U datasets==2.17.0

%pip install --upgrade pip
%pip install --disable-pip-version-check \
    torch==1.13.1 \
    torchdata==0.5.1 --quiet

%pip install \
    transformers==4.27.2 \
    evaluate==0.4.0 \
    rouge_score==0.1.2 \
    loralib==0.1.1 \
    peft==0.3.0 --quiet

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [ ]:
original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

### 1.3 - Test the Model with Zero Shot Inferencing

In [ ]:
index = 200

dialogue = mydataset['test'][index]['dialogue']
summary = mydataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

<a name='2'></a>
## 2 - Perform Full Fine-Tuning

In [ ]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example["dialogue"]]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
tokenized_datasets = mydataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

In [ ]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

### 2.2 - Fine-Tune the Model with the Preprocessed Dataset

In [ ]:
output_dir = f'models/vul-fix-training-{str(int(time.time()))}'

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()

In [ ]:
# Save the trained model
trainer.save_model(f'models/instruct_model-{output_dir}')

In [ ]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained(f'models/instruct_model-{output_dir}', torch_dtype=torch.bfloat16)

### 2.3 - Evaluate the Model Qualitatively (Human Evaluation)

In [ ]:
index = 200
dialogue = mydataset['test'][index]['dialogue']
human_baseline_summary = mydataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors="pt").input_ids

original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{human_baseline_summary}')
print(dash_line)
print(f'ORIGINAL MODEL:\n{original_model_text_output}')
print(dash_line)
print(f'INSTRUCT MODEL:\n{instruct_model_text_output}')

### 2.4 - Evaluate the Model Quantitatively (with ROUGE Metric)

In [ ]:
rouge = evaluate.load('rouge')

In [ ]:
dialogues = mydataset['test'][0:10]['dialogue']
human_baseline_summaries = mydataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []

for _, dialogue in enumerate(dialogues):
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary: """
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids

    original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_model_text_output = tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_model_text_output)

    instruct_model_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_model_text_output)
    
zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))
 
df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'instruct_model_summaries'])
df

In [ ]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

In [ ]:
import evaluate
import pandas as pd

results = pd.read_csv("log/result-2024-05-26--12-05.csv")

bleu = evaluate.load('bleu')

bleu.compute(predictions=results['instruct_model_fixes'], references=results['original_model_fixes'])

In [2]:
from codebleu import calc_codebleu
import pandas as pd

def calc_codebleu_score(references, predictions, lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None):
    """ 
    Calculate the CodeBLEU score for the given references and predictions. 
    Args: 
    references (List[str]): List of reference code snippets. 
    predictions (List[str]): List of predicted code snippets. 
    lang (str): Programming language of the code snippets. Default is 'python'. 
    weights (Tuple[float]): Weights for 1-gram, 2-gram, 3-gram, and 4-gram. Default is (0.25, 0.25, 0.25, 0.25). 
    tokenizer (Optional): Tokenizer to use for tokenization. Default is None. 
    Returns: Dict[str, float]: Dictionary containing the CodeBLEU score and its components. 
    """ 
    # Calculate the CodeBLEU score
    result = calc_codebleu(references, predictions, lang=lang, weights=weights, tokenizer=tokenizer)
    return result

results = pd.read_csv("log/result-2024-05-26--12-05.csv")

baselines = results['human_baseline_fixes'].tolist()
references = results['original_model_fixes'].tolist()
predictions = results['instruct_model_fixes'].tolist()
program_langs = results['programming_language'].tolist()

# Call the calc_codebleu_score function
base_scores = calc_codebleu_score(baselines, references) 
instruct_scores = calc_codebleu_score(baselines, predictions)

improved_performance = {key: instruct_scores[key] - base_scores[key] 
                        for key in instruct_scores.keys()}

keys = base_scores.keys() & instruct_scores.keys()

# Calculate the improvement as a percentage for each key
improved_performance_percent = {key: (instruct_scores[key] - base_scores[key]) * 1.0 / base_scores[key] if base_scores[key] != 0 else 0 for key in keys}

# Convert percentage to easier-to-read format
improved_performance_percent = {key: f"{improvement * 100:.2f}%" for key, improvement in improved_performance_percent.items()}

# put these in a pandas dataframe
df = pd.DataFrame([base_scores, instruct_scores], index=['Base', 'Instruct']).T
df['Base'] = df['Base']*100
df['Instruct'] = df['Instruct']*100
df['Improvement'] =  df['Instruct'] - df['Base']

# round to 2 decimal places
df = df.round(2)
df

/Users/guru/miniconda/envs/guesslang/lib/python3.8/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


,Base,Instruct,Improvement
codebleu,8.46,36.06,27.60
ngram_match_score,0.02,16.30,16.28
weighted_ngram_match_score,0.12,18.20,18.08
syntax_match_score,25.35,68.08,42.72
dataflow_match_score,8.33,41.67,33.33


In [3]:
from guesslang import Guess

# define function with try catch for guess lang
def guess_lang(code):
    try:
        guess = Guess()
        return guess.language_name(code)
    except:
        return "c"

langs = [guess_lang(x) for x in baselines]
langs

['Shell', 'Shell', 'PHP', 'SQL']

In [1]:
from codebleu import calc_codebleu
import pandas as pd
from tabulate import tabulate

def calc_codebleu_scores(references, predictions, langs, weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None):
    """ 
    Calculate the CodeBLEU scores.
    
    Args:
        references (List[str]): List of reference code strings.
        predictions (List[str]): List of predicted code strings.
        langs (List[str]): List of programming languages for each pair.
        weights (Tuple[float]): Weights for 1-gram, 2-gram, 3-gram, 4-gram. Default is (0.25, 0.25, 0.25, 0.25). 
        tokenizer (Optional): Tokenizer to use for tokenization. Default is None.
        
    Returns: 
        List[Dict[str, float]]: List of dictionaries, each containing the CodeBLEU score and its components for a 
        corresponding pair of reference and prediction. 
    """ 
    scores = [
        calc_codebleu([ref], [pred], lang=lg, weights=weights, tokenizer=tokenizer)
        for ref, pred, lg in zip(references, predictions, langs)
    ]
    return scores

def show_bleu_scores(original_bleu_scores, instruct_bleu_scores):
    """ 
    Display the improvement in BLEU scores.
    
    Args:
        original_bleu_scores (List[Dict[str, float]]): BLEU scores for the original model.
        instruct_bleu_scores (List[Dict[str, float]]): BLEU scores for the instruct model.
    """
    df_original = pd.DataFrame(original_bleu_scores).mean()
    df_instruct = pd.DataFrame(instruct_bleu_scores).mean()
    
    df = pd.concat([df_original, df_instruct], axis=1)
    df.columns = ['Base', 'Instruct']
    df['Base'] *= 100
    df['Instruct'] *= 100
    df['Improvement'] = df['Instruct'] - df['Base']
    df = df.round(2).applymap(lambda x: f"{x:.2f}%")
    
    print(f"Weighted average BLEU scores improved: \n{tabulate(df, headers='keys', tablefmt='psql')}")


def evaluate_bleu(results):
    try: 
        human_baseline_fixes = results['human_baseline_fixes'].tolist()
        original_model_fixes = results['original_model_fixes'].tolist()
        instruct_model_fixes = results['instruct_model_fixes'].tolist()
        langs = results['programming_language'].str.lower().replace('c++', 'cpp').tolist()

        original_bleu_scores = calc_codebleu_scores(
            references=human_baseline_fixes, 
            predictions=original_model_fixes, 
            langs=langs
        )

        instruct_bleu_scores = calc_codebleu_scores(
            references=human_baseline_fixes, 
            predictions=instruct_model_fixes, 
            langs=langs
        )
        
        show_bleu_scores(original_bleu_scores, instruct_bleu_scores)
    except FileNotFoundError:
        print(f"File {csv_file} not found.")
    except pd.errors.EmptyDataError:
        print(f"No data in {csv_file}.")
    except pd.errors.ParserError:
        print(f"Error parsing {csv_file}.")
    except Exception as e:
        print(f"An error occurred: {str(e)}")

# Load CSV data
csv_file = 'log/result-2024-05-26--12-05.csv'
results = pd.read_csv(csv_file)
results['programming_language'] = 'c'


evaluate_bleu(results)


Weighted average BLEU scores improved: 
+----------------------------+--------+------------+---------------+
|                            | Base   | Instruct   | Improvement   |
|----------------------------+--------+------------+---------------|
| codebleu                   | 2.11%  | 25.77%     | 23.66%        |
| ngram_match_score          | 0.00%  | 6.54%      | 6.54%         |
| weighted_ngram_match_score | 0.06%  | 16.90%     | 16.84%        |
| syntax_match_score         | 2.65%  | 49.62%     | 46.97%        |
| dataflow_match_score       | 5.72%  | 30.01%     | 24.29%        |
+----------------------------+--------+------------+---------------+


/Users/guru/miniconda/envs/fixme/lib/python3.10/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [6]:
results['programming_language'].str.lower().replace('c++', 'cpp').tolist()

['c', 'c', 'c', 'c']

In [ ]:
import nltk.translate.bleu_score as bleu_score
import pandas as pd

results = pd.read_csv("log/result-2024-05-26--12-05.csv")

list_of_references = results['original_model_fixes']
candidates = results['instruct_model_fixes']

# bleu = bleu_score.corpus_bleu(list_of_references, candidates)
# print(bleu)
result = calc_codebleu(list_of_references, candidates, lang="python", weights=(0.25, 0.25, 0.25, 0.25), tokenizer=None)
print(result)

In [ ]:
len(results)

In [ ]:


human_baseline_summaries = results['human_baseline_summaries'].values
original_model_summaries = results['original_model_summaries'].values
instruct_model_summaries = results['instruct_model_summaries'].values

original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('INSTRUCT MODEL:')
print(instruct_model_results)

In [ ]:
print("Absolute percentage improvement of INSTRUCT MODEL over ORIGINAL MODEL")

improvement = (np.array(list(instruct_model_results.values())) - np.array(list(original_model_results.values())))
for key, value in zip(instruct_model_results.keys(), improvement):
    print(f'{key}: {value*100:.2f}%')

In [1]:
import pandas as pd 
from tabulate import tabulate

original_model_results = {'rouge1': 0.21050637958532695, 'rouge2': 0.08698568872987478, 'rougeL': 0.17851984759879497, 'rougeLsum': 0.20845959595959596}
instruct_model_results = {'rouge1': 0.21016899766899766, 'rouge2': 0.08698568872987478, 'rougeL': 0.17818246568246568, 'rougeLsum': 0.20845959595959596}


def show_rouge_scores(original_model_results, instruct_model_results):
    df = pd.DataFrame([original_model_results, instruct_model_results], index=['Base', 'Instruct']).T
    df['Base'] = df['Base']*100
    df['Instruct'] = df['Instruct']*100
    df['Improvement'] =  df['Instruct'] - df['Base']
    df = df.round(2).applymap(lambda x: f"{x:.2f}%")
    print(f"The ROUGE scores improved: \n{tabulate(df, headers='keys', tablefmt='psql')}")

show_rouge_scores(original_model_results, instruct_model_results)

The ROUGE scores improved: 
+-----------+--------+------------+---------------+
|           | Base   | Instruct   | Improvement   |
|-----------+--------+------------+---------------|
| rouge1    | 21.05% | 21.02%     | -0.03%        |
| rouge2    | 8.70%  | 8.70%      | 0.00%         |
| rougeL    | 17.85% | 17.82%     | -0.03%        |
| rougeLsum | 20.85% | 20.85%     | 0.00%         |
+-----------+--------+------------+---------------+
